## Preprocessing

In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import save_model,load_model
from google.colab import drive


#  Import PANDAS
import pandas as pd


In [3]:
# constants
GDRIVE_PATH = '/content/gdrive/My Drive'
TRAINED_MODEL_PATH = f"{GDRIVE_PATH}/deep-learning-challenge/model.h5"


In [4]:
# read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop("EIN", axis = 1 , inplace = True)
application_df.drop("NAME", axis = 1 , inplace = True)
application_df.drop("STATUS", axis = 1 , inplace = True)

application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,N,142590,1


In [6]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace=["T9","T13","T12","T2","T25","T14","T29","T15","T17"]

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
application_df["CLASSIFICATION"].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts()>1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classification_cutoff = application_df["CLASSIFICATION"].value_counts()[application_df["CLASSIFICATION"].value_counts()<= 1883]
classification_cutoff = classification_cutoff.index
classifications_to_replace = classification_cutoff.tolist()
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     4144
C3000     1918
Name: CLASSIFICATION, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
X=pd.get_dummies(application_df)
X

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,5000,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,108590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,6692,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,142590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,5000,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,5000,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,5000,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [13]:
# Split our preprocessed data into our features and target arrays
y=application_df['IS_SUCCESSFUL']
X=X.drop(columns='IS_SUCCESSFUL')


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split (X,y,random_state=1)
X_train.shape

(25724, 41)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# try 1
nn_model = tf.keras.models.Sequential()

# Input layer
nn_model.add(tf.keras.layers.Dense(units=100,activation="relu",input_dim=41))

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=30,activation="sigmoid"))
# second hidden layer
nn_model.add(tf.keras.layers.Dense(units=10,activation="sigmoid"))


# Output layer
nn_model.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))
# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 100)               4200      
                                                                 
 dense_5 (Dense)             (None, 30)                3030      
                                                                 
 dense_6 (Dense)             (None, 10)                310       
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                                 
Total params: 7,551
Trainable params: 7,551
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# try 2
nn_model = tf.keras.models.Sequential()

# Input layer
nn_model.add(tf.keras.layers.Dense(units=100,activation="relu",input_dim=41))

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=50,activation="sigmoid"))
# second hidden layer
nn_model.add(tf.keras.layers.Dense(units=50,activation="sigmoid"))


# Output layer
nn_model.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))
# Check the structure of the model
nn_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               4200      
                                                                 
 dense_9 (Dense)             (None, 50)                5050      
                                                                 
 dense_10 (Dense)            (None, 50)                2550      
                                                                 
 dense_11 (Dense)            (None, 1)                 51        
                                                                 
Total params: 11,851
Trainable params: 11,851
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# try 3
nn_model = tf.keras.models.Sequential()

# Input layer
nn_model.add(tf.keras.layers.Dense(units=100,activation="relu",input_dim=41))

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=40,activation="relu"))
# second hidden layer
nn_model.add(tf.keras.layers.Dense(units=40,activation="relu"))


# Output layer
nn_model.add(tf.keras.layers.Dense(units=1,activation="sigmoid"))
# Check the structure of the model
nn_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 100)               4200      
                                                                 
 dense_13 (Dense)            (None, 40)                4040      
                                                                 
 dense_14 (Dense)            (None, 40)                1640      
                                                                 
 dense_15 (Dense)            (None, 1)                 41        
                                                                 
Total params: 9,921
Trainable params: 9,921
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [20]:
# Train the model
nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.5868 - accuracy: 0.7128
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5633 - accuracy: 0.7272
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5595 - accuracy: 0.7275
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5568 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5553 - accuracy: 0.7304
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5543 - accuracy: 0.7296
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5533 - accuracy: 0.7315
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5526 - accuracy: 0.7308
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5520 - accuracy: 0.7313
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5510 - accura

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5581 - accuracy: 0.7269 - 494ms/epoch - 2ms/step
Loss: 0.5580726265907288, Accuracy: 0.7268804907798767


In [22]:
# Export our model to HDF5 file
save_model(nn_model, TRAINED_MODEL_PATH,"AlphabetSoupCharity_Optimization")

In [ ]:
nn_model_loaded = load_model( TRAINED_MODEL_PATH )

In [ ]:
nn_model_loaded.evaluate(X_test_scaled,y_test,verbose=2)

268/268 - 0s - loss: 0.5630 - accuracy: 0.7276 - 452ms/epoch - 2ms/step


[0.5629809498786926, 0.727580189704895]